## Transformacion
 En este cuaderno realizamos la transformacion el los datos, usando los csv extraidos creamos un data frame con la informacion de los csv

### .


In [ ]:
import os
import numpy as np
import pandas as pd
import requests
from time import sleep
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

In [ ]:
shows = pd.read_csv("01_shows_master.csv")
eps = pd.read_csv("02_episodios_raw.csv")

In [ ]:
shows.head(2)

In [ ]:
eps.head(2)

In [ ]:
#Convertir fechas
eps["air_date"] = pd.to_datetime(eps["air_date"], errors="coerce")
shows["start_date"] = pd.to_datetime(shows["start_date"], errors="coerce")

In [ ]:
eps_min = eps.groupby("show_id", as_index=False)["air_date"].min()
eps_min.rename(columns={"air_date": "episode_date"}, inplace=True)
eps_min["episode_date"] = eps_min["episode_date"].dt.strftime("%Y-%m-%d")

eps_min.head(2)

In [ ]:
eps_min = eps.groupby("show_id", as_index=False)["air_date"].min()
eps_min.rename(columns={"air_date": "episode_date"}, inplace=True)

# Mantener solo la fecha como tipo datetime.date
eps_min["episode_date"] = eps_min["episode_date"].dt.date

eps_min.head(2)

In [ ]:
df = pd.DataFrame(data = shows, columns = ["id", "name", "status", "start_date", "end_date", "network", "country", "runtime", "rating", "rating_count","genres","total_episodes"])

df

In [ ]:
df1 = shows[shows["id"]].drop_duplicates()

In [ ]:
eps.head(100)

In [ ]:
df2 = eps[eps["show_id"] == 6320]

In [ ]:
eps_show_id = eps.groupby(by = "show_id", as_index = False).agg({"season" : "max"})
eps_show_id.head()

### Comparar fechas de inicio
Una vez que las columnas están en formato datetime (solo la fecha)
Del dataframe de episodios, hacer un groupby por la columna de id y quedarme con el elemento más pequeño de la columna de fecha (min)
Luego comparar ambas columnas del dataframe nuevo y el dataframe de shows. Quedarse con la más pequeña.

### Total de episodios por año
Del df de episodios, crear la columna año (year) y hacer groupby sobre la columna year y contar cualquier otra columna, puede ser la columna de id_show, puede ser con un value count, si es para una gráfica, lo mejor sería un groupby y ordenar por año

In [ ]:
df_unido = pd.merge(left=shows, right=eps_show_id,left_on='id',right_on='show_id',how='left')
df_unido.head()

In [ ]:
df_unido[~(df_unido["rating"] == 0)].head()

,id,name,status,start_date,end_date,network,country,runtime,rating,rating_count,genres,total_episodes,show_id,season
0,75924,Partner Track,Ended,2022-08-26,NaN,Netflix,US,10,5.5000,2,Drama,10,75924.0,1.0
6,75930,I Am Shauna Rae,To Be Determined,2022-01-11,NaN,TLC,US,60,10.0000,1,NaN,16,75930.0,2.0
13,75937,RuPaul's Drag Race UK vs The World,Running,2022-01-17,NaN,BBC Three,GB,10,7.0000,1,NaN,14,75937.0,2.0
53,75978,The Dawn of the Witch,Running,2022-04-07,NaN,TBS,JP,24,10.0000,3,"Comedy, Adventure, Fantasy, Anime",12,75978.0,1.0
57,75982,Call of the Night,Running,2022-07-07,NaN,Fuji TV,JP,30,9.6667,6,"Comedy, Fantasy, Anime, Romance",25,75982.0,2.0


In [ ]:
# DataFrame de generos por show
df_genres = pd.DataFrame(data = df_unido[["id", "genres"]], columns = ["id", "genres"])

df_genres["genres"] = df_genres["genres"].apply(lambda x : x.split(", ") if not pd.isna(x) else x)

df_genres = df_genres.explode("genres")

df_genres.shape

In [ ]:
df_genres.value_counts("genres")

In [ ]:
df_unido.value_counts("rating", normalize = True)*100

In [ ]:
eps.info()

shows.info()

df_unido.info()

In [ ]:
#primeros 5
df_unido.head()

In [ ]:
#ultimos 5
df_unido.tail()

In [ ]:
# Calcula la media, la mediana y la desviación estandar para cada columna numérica.

print(f'Columna: runtime, Media: {df_unido["runtime"].mean()}, Median: {df_unido["runtime"].median()}, std: {df_unido["runtime"].std()}')
print(f'Columna: rating, Media: {df_unido["rating"].mean()}, Median: {df_unido["rating"].median()}, std: {df_unido["rating"].std()}')
print(f'Columna: rating_count, Media: {df_unido["rating_count"].mean()}, Median: {df_unido["rating_count"].median()}, std: {df_unido["rating_count"].std()}')
print(f'Columna: total_episodes, Media: {df_unido["total_episodes"].mean()}, Median: {df_unido["total_episodes"].median()}, std: {df_unido["total_episodes"].std()}')
print(f'Columna: season, Media: {df_unido["season"].mean()}, Median: {df_unido["season"].median()}, std: {df_unido["season"].std()}')


In [ ]:
columnas_numericas = ["runtime", "rating", "rating_count", "total_episodes", "season"]

df_unido[columnas_numericas].mean()

In [ ]:
df_unido[columnas_numericas].median()

In [ ]:
df_unido[columnas_numericas].std()

In [ ]:
len(df_unido)

In [ ]:
# mayor runtime
df_unido.sort_values("runtime", ascending = False).head(1)

In [ ]:
# menor runtime
df_unido.sort_values("runtime").head(1)

In [ ]:
# mayor rating
df_unido.sort_values("rating", ascending = False).head(1)

In [ ]:
# menor rating
df_unido.sort_values("rating").head(1)

In [ ]:
# tiene mas episodios
df_unido.sort_values("total_episodes", ascending = False).head(1)

In [ ]:
# tiene menos episodios como saco tos los qeu son 0
df_unido.sort_values("total_episodes").head(1)

In [ ]:
# tiene mas episodios y rating
df_unido.sort_values(["rating", "total_episodes"], ascending = False).head(1)

In [ ]:
# tiene runtimes y rating
df_unido.sort_values(["runtime", "rating"], ascending = False).head(1)

In [ ]:
df_unido.describe()

In [ ]:
df_unido["genres"].value_counts()

In [ ]:
df_unido["network"].value_counts()


In [ ]:
df_unido["country"].value_counts()

In [ ]:
df_unido.head()